In [1]:
import os
print(os.getcwd())

import warnings
warnings.filterwarnings("ignore", message="FP16 is not supported on CPU*")

/home/kruu/git_folder/atc_clearences


****
# Voice Activity Detection
****

In [4]:
import wave
import webrtcvad
from pydub import AudioSegment

audio_path = "data/LSZH1-Twr-May-10-2025-1100Z.mp3"
output_dir = "data/speech_segments_1100"
wav_path = "notebooks/temp_audio.wav"
os.makedirs(output_dir, exist_ok=True)

# Convert MP3 to WAV
audio = AudioSegment.from_file(audio_path).set_channels(1).set_frame_rate(16000)
audio.export(wav_path, format="wav")

# Read WAV file as bytes
# Minimum duration = 1 second = 16000 samples for 16kHz

min_duration_samples = 16000  # 1 second
with wave.open(wav_path, 'rb') as wf:
    vad = webrtcvad.Vad(2)  # Sensitivity: 0 (low), 3 (high)
    frame_duration = 30  # ms
    sample_rate = wf.getframerate()
    frame_bytes = int(sample_rate * frame_duration / 1000) * 2  # 16-bit samples
    frames = wf.readframes(wf.getnframes())

    chunks = []
    i = 0
    voiced = False
    chunk = b''
    timestamp = 0
    for offset in range(0, len(frames), frame_bytes):
        frame = frames[offset:offset + frame_bytes]
        if len(frame) < frame_bytes:
            break
        is_speech = vad.is_speech(frame, sample_rate)
        if is_speech:
            if not voiced:
                voiced = True
                chunk = b''
            chunk += frame
        else:
            if voiced:
                voiced = False
                if len(chunk) >= min_duration_samples * 2:  # 2 bytes per sample (16-bit audio)
                    fname = os.path.join(output_dir, f"speech_{i:03d}.wav")
                    with wave.open(fname, 'wb') as out:
                        out.setnchannels(1)
                        out.setsampwidth(2)
                        out.setframerate(sample_rate)
                        out.writeframes(chunk)
                    i += 1

**** 
# Whisper
****

In [5]:
import whisper

model = whisper.load_model("turbo")
for f in sorted(os.listdir(output_dir)):
    if f.endswith(".wav"):
        result = model.transcribe(os.path.join(output_dir, f), language="en")
        print(f"{f}: {result['text']}")

speech_000.wav:  ...for the release, 4 knots from the 2-8 cleared for takeoff.
speech_001.wav:  runway 28, clip takeoff, phase 458, takeoff
speech_002.wav:  The flight Yankee Vita Lana Pranich Week.
speech_003.wav:  wäre at the Brombe though, fill in half an Ear utilizblematica!
speech_004.wav:  The Fat Lima Delta tower green, Cigand Embraer ahead, line up running to a delay behind.
speech_005.wav:  MAT shorten some scenario.
speech_006.wav:  The 242 contact departure, Rade.
speech_007.wav:  Adé, two for two.
speech_008.wav:  I'm gonna go on to the final lift now.
speech_009.wav:  Rega 1, 2, lander, no discretion, fix change approved. Bis nachher.
speech_010.wav: 和海の綺麗の2020年
speech_011.wav:  The 2.5 ADECO contact departure RAD. departure RAD, response, HADECO.
speech_012.wav:  Despite Yankee Victor, wind 3233 is 4 knots running 2H lead for takeoff.
speech_013.wav:  We're now at 1004.5, lions, wreckage with En
speech_014.wav:  We 5, Yankee-Victor, contact departure. Departure, address i